In [1]:
from xbbg import blp
from xbbg.core import conn,process
from datetime import date
from datetime import datetime

import pandas as pd

In [2]:
SAVEDATA = True
DATE_PREFED = '2025-01-26'
DATE_HISTORIC = '2025-01-31'

# STIR Futures

In [3]:
def get_futures_chain(fut_active,Nchain):
    chain = blp.bds(fut_active,'FUT_CHAIN')
    futflds = ['PX_SETTLE_LAST_DT','PX_LAST','OPEN_INT','FUT_DAYS_EXPIRE','LAST_TRADEABLE_DT']
    ticks = chain['security_description'].iloc[:Nchain]
    futdata = blp.bdp(ticks, futflds).sort_values('last_tradeable_dt')
    return futdata

### Get Live Data

In [4]:
fut = 'FFA Comdty'
Nchain = 14
ffdata = get_futures_chain(fut,Nchain)
ffdata.index.name = 'ticker'

DATE = ffdata['px_settle_last_dt'].iloc[0].strftime('%Y-%m-%d')

### Get Historic Date

In [5]:
futflds = ['PX_LAST']
chain = blp.bds(fut,'FUT_CHAIN')
ticks = chain['security_description'].iloc[:Nchain]
futdata_historic = blp.bdh(ticks, futflds, start_date=DATE_HISTORIC).droplevel(level=1,axis=1).iloc[[0],:].T

futdata_historic.index.name = 'ticker'
futdata_historic.columns= ['px_last']
futdata_historic['last_tradeable_dt'] = ffdata['last_tradeable_dt'].copy()

### Get PreFed Date

In [6]:
futflds = ['PX_LAST']
chain = blp.bds(fut,'FUT_CHAIN')
ticks = chain['security_description'].iloc[:Nchain]
futdata_prefed = blp.bdh(ticks, futflds, start_date=DATE_PREFED).droplevel(level=1,axis=1).iloc[[0],:].T

futdata_prefed.index.name = 'ticker'
futdata_prefed.columns= ['px_last']
futdata_prefed['last_tradeable_dt'] = ffdata['last_tradeable_dt'].copy()

### Fed Funds

In [7]:
DATES = [DATE_HISTORIC,DATE_PREFED,DATE]
fedfunds = pd.DataFrame(index=DATES,columns=['fed funds rate'],dtype=float)
for dt in DATES:
    fedfunds.loc[dt,:] = blp.bdp(['FEDL01 Index'],flds=['PX_LAST'],single_date_override=dt).values
fedfunds.index.name = 'date'
fedfunds.sort_index(inplace=True)

In [8]:
fedfunds

,fed funds rate
date,
2025-01-26,4.33
2025-01-31,4.33
2025-02-25,4.33


### Fed Calendar

In [9]:
feddts = [                                        
          '2025-03-19',
          '2025-05-07',
          '2025-06-18',
          '2025-07-30',
          '2025-09-17',
          '2025-10-29',
          '2025-12-10',
          '2026-01-29',
          '2026-03-18'
         ]

feddts = pd.Series([datetime.strptime(dt,'%Y-%m-%d').date() for dt in feddts], name='meeting dates')

***

## SOFR Futures

In [10]:
fut = 'SERA Comdty'
Nchain = 12
sofrdata1 = get_futures_chain(fut,Nchain)
sofrdata1

,px_settle_last_dt,px_last,open_int,fut_days_expire,last_tradeable_dt
SERG5 Comdty,2025-02-25,95.6575,429788,2,2025-02-28
SERH5 Comdty,2025-02-25,95.68,303807,33,2025-03-31
SERJ5 Comdty,2025-02-25,95.69,233503,63,2025-04-30
SERK5 Comdty,2025-02-25,95.73,242859,93,2025-05-30
SERM5 Comdty,2025-02-25,95.78,156385,124,2025-06-30
SERN5 Comdty,2025-02-25,95.86,111678,155,2025-07-31
SERQ5 Comdty,2025-02-25,95.92,45680,184,2025-08-29
SERU5 Comdty,2025-02-25,95.96,39351,216,2025-09-30
SERV5 Comdty,2025-02-25,96.035,21854,247,2025-10-31
SERX5 Comdty,2025-02-25,96.09,13386,275,2025-11-28


In [11]:
fut = 'SFRA Comdty'
Nchain = 12
sofrdata3 = get_futures_chain(fut,Nchain)
sofrdata3

,px_settle_last_dt,px_last,open_int,fut_days_expire,last_tradeable_dt
SFRZ4 Comdty,2025-02-25,95.6375,1034591,20,2025-03-18
SFRF5 Comdty,2025-02-25,95.655,5989,48,2025-04-15
SFRG5 Comdty,2025-02-25,95.675,2247,83,2025-05-20
SFRH5 Comdty,2025-02-25,95.695,1278807,111,2025-06-17
SFRJ5 Comdty,2025-02-25,95.75,2077,139,2025-07-15
SFRK5 Comdty,2025-02-25,95.815,2471,174,2025-08-19
SFRM5 Comdty,2025-02-25,95.865,1169077,202,2025-09-16
SFRN5 Comdty,2025-02-25,95.94,141,230,2025-10-14
SFRQ5 Comdty,2025-02-25,96.01,417,265,2025-11-18
SFRU5 Comdty,2025-02-25,96.04,855558,293,2025-12-16


### Save Data

In [12]:
if SAVEDATA:    
    outfile = f'../data/fedfutures_{DATE}.xlsx'
    with pd.ExcelWriter(outfile) as writer:  
        ffdata.to_excel(writer, sheet_name= f'{DATE}', index=True)         
        futdata_historic.to_excel(writer, sheet_name= f'{DATE_HISTORIC}', index=True)
        futdata_prefed.to_excel(writer, sheet_name= f'{DATE_PREFED}', index=True)

        fedfunds.to_excel(writer, sheet_name= 'fed funds', index=True) 
        feddts.to_excel(writer, sheet_name= 'fed meetings', index=False)


26FEB2025_10:09:41.385 19512:6202650624 WARN apism_apisession.cpp:1326 Apisession.aliveCheck Alive Request Retries exhausted (2) on [localhost:8194 -> 127.0.0.1:51990] 

26FEB2025_10:09:41.389 19512:6204321792 WARN blpapi_platformcontroller.cpp:489 blpapi.session.{1}.platformcontroller Connectivity lost, no connected endpoints. 

26FEB2025_10:09:49.087 19512:6203224064 WARN blpapi_apicmadapter.cpp:341 blpapi.session.{1}.transporttcp.apicmadapter Connection attempt timed out localhost:8194 <127.0.0.1:8194> 

26FEB2025_10:09:49.087 19512:6203224064 WARN blpapi_platformtransporttcp.cpp:138 blpapi.session.{1}.transporttcp<0><localhost:8194> 127.0.0.1, session pool state=Failed 

26FEB2025_10:12:51.673 19512:6203224064 WARN blpapi_apicmadapter.cpp:341 blpapi.session.{1}.transporttcp.apicmadapter Connection attempt timed out localhost:8194 <127.0.0.1:8194> 

26FEB2025_10:12:51.673 19512:6203224064 WARN blpapi_platformtransporttcp.cpp:138 blpapi.session.{1}.transporttcp<0><localhost:8194> 12